# Introduction

In this notebook, we will look at how hierarchical models can help us in scenarios where we do not have sufficient data to meaningfully quantify our uncertainties. By making assumptions about relatedness, we can borrow statistical power from other observations, 

In [ ]:
import pandas as pd
import janitor as jn
import pymc3 as pm

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_csv('../data/finch_beaks_2012.csv')
df = (jn.DataFrame(df)
      .clean_names()
      .rename_column('blength', 'beak_length')
      .rename_column('bdepth', 'beak_depth')
      .encode_categorical('species')
     )
df.groupby('species').size()

In [ ]:
df.groupby('species')['beak_depth'].describe()

In [ ]:
fortis_filter = df['species'] == 'fortis'
scandens_filter = df['species'] == 'scandens'
unknown_filter = df['species'] == 'unknown'

In [ ]:
def enc_species(x):
    if x == 'fortis':
        return 0
    if x == 'scandens':
        return 1
    else:
        return 2

In [ ]:
df['species_enc'] = df['species'].apply(lambda x: enc_species(x))
df['species_enc'].head(5)

In [ ]:
with pm.Model() as beak_depth_model:
    # SD can only be positive, therefore it is reasonable to constrain to >0
    # Likewise for betas.
    sd_hyper = pm.HalfCauchy('sd_hyper', beta=100)
    beta_hyper = pm.HalfCauchy('beta_hyper', beta=100)
    
    # Beaks cannot be of "negative" mean, therefore, HalfNormal is 
    # a reasonable, constrained prior.
    mean = pm.HalfNormal('mean', sd=sd_hyper, shape=(3,))
    sd = pm.HalfCauchy('sd', beta=beta_hyper, shape=(3,))
    nu = pm.Exponential('nu', lam=1/29.) + 1
    like = pm.StudentT('likelihood', 
                       nu=nu,
                       mu=mean[df['species_enc']], 
                       sd=sd[df['species_enc']], 
                       observed=df['beak_depth'])

In [ ]:
with beak_depth_model:
    trace = pm.sample(2000, nuts_kwargs={'target_accept': 0.95})

In [ ]:
pm.traceplot(trace, varnames=['mean'])

In [ ]:
pm.plot_posterior(trace, varnames=['mean'])